In [1]:
import networkx as nx
import pandas as pd
import numpy as np
from utils import load_data,normalize,doublerelu
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
torch.set_printoptions(sci_mode=False)
import time

In [2]:
import warnings
warnings.filterwarnings("ignore") 

In [3]:
cuda = torch.cuda.is_available()
weight_decay = 10e-4
epochs = 10000
seed = 165
hidden = 10
lr = 0.00001

In [4]:
np.random.seed(seed)
torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)

In [5]:
class GNN1Layer(Module):

    def __init__(self, batch_size, in_features, out_features):
        super(GNN1Layer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.batch_size = batch_size

        weight1_eye = torch.FloatTensor(torch.eye(in_features, out_features))
        weight1_eye = weight1_eye.reshape((1, in_features, out_features))
        weight1_eye = weight1_eye.repeat(batch_size, 1, 1)
        self.weight1 = Parameter(weight1_eye)
        self.weight2 = Parameter(torch.zeros(batch_size, in_features, out_features))

    def forward(self, input, adj):
        v1 = torch.bmm(input, self.weight1)
        v2 = torch.bmm(torch.bmm(adj, input), self.weight2)
        output = v1 + v2
        return output

In [6]:
class GNN2Layer(Module):

    def __init__(self, batch_size, in_features, out_features):
        super(GNN2Layer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.batch_size = batch_size
        weight1_eye = torch.FloatTensor(torch.eye(in_features, out_features))
        weight1_eye = weight1_eye.reshape((1, in_features, out_features))
        weight1_eye = weight1_eye.repeat(batch_size, 1, 1)
        weight1_rand = torch.empty(batch_size,in_features,out_features-in_features)
        torch.nn.init.xavier_uniform_(weight1_rand, gain=1.0)
        self.weight1 = Parameter(weight1_eye)
        self.weight2 = Parameter(torch.zeros(batch_size, in_features, out_features))

    def forward(self, input, adj):
        v1 = torch.bmm(input, self.weight1)
        v2 = torch.bmm(torch.bmm(adj, input), self.weight2)
        output = v1 + v2
        return output

In [7]:
class GNN1(nn.Module):

    def __init__(self, batch_size, nfeat, ndim, hidden):
        super(GNN1, self).__init__()

        self.gc1 = GNN1Layer(batch_size, ndim, ndim)

    def forward(self, x, adj):
        x = doublerelu(self.gc1(x, adj))
        x = x/x.sum(axis=2).unsqueeze(2) #normalize st sum = 1
        return x

In [8]:
class GNN2(nn.Module):

    def __init__(self, batch_size, nfeat, ndim, hidden):
        super(GNN2, self).__init__()

        self.gc1 = GNN2Layer(batch_size, ndim, hidden)
        self.gc2 = GNN1Layer(batch_size, hidden, ndim)

    def forward(self, x, adj):
        x = doublerelu(self.gc1(x, adj))
        x = doublerelu(self.gc2(x, adj))
        x = x/x.sum(axis=2).unsqueeze(2) #normalize st sum = 1
        return x

In [9]:
def train(adj,feature,labels,mask_percentage):
    
    adj_norm = normalize(adj)
    
    feature = feature - 1
    nb_label = int(max(feature))+1
    features = np.eye(nb_label)[np.array(feature,dtype=int).reshape(1,-1)]

    # Masking
    number_of_rows = features[0].shape[0]
    random_indices = np.random.choice(number_of_rows, size=int(mask_percentage*number_of_rows), replace=False)
    random_rows = features[0][random_indices, :]
    features[0][random_indices, :] = np.tile(np.array([[0.2]]),random_rows.shape)
    
    labels = labels - 1
    
    adj = torch.FloatTensor(adj)
    adj_norm = torch.FloatTensor(adj_norm)
    features = torch.FloatTensor(features)
    labels = torch.FloatTensor(labels)
    
    model = GNN1(batch_size=adj.shape[0],
                nfeat=adj.shape[1],
                ndim=nb_label,
                hidden=hidden)
    if cuda:
        model.cuda()
        features = features.cuda()
        adj = adj.cuda()
        adj_norm = adj_norm.cuda()
        labels = labels.cuda()
    
    # Train model
    t_total = time.time()

    optimizer = optim.Adam(model.parameters(),
                           lr=lr, weight_decay=weight_decay)
    
    criterion = nn.CrossEntropyLoss()
    
    
    for epoch in range(epochs):

        t = time.time()
        model.train()
        optimizer.zero_grad()

        output = model(features, adj_norm)
        
        loss = criterion(output[0],labels.reshape(-1).long())

        loss.backward(retain_graph=True)

        optimizer.step()

        if epoch == 0:
            best_loss = loss
            best_output = output
        else:
            if loss < best_loss:
                best_loss = loss
                best_output = output

        if epoch % 1000 == 0:
            print('Epoch: {:04d}'.format(epoch + 1),
                  'Loss: {:.8f}'.format(loss.item()),
                  'time: {:.4f}s'.format(time.time() - t))
            
    print("Optimization Finished!")
    print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
    
    return best_loss,output

In [10]:
adj,feature,labels = load_data()

In [12]:
mask_percentage = [0.3,0.5,0.7]
for m in mask_percentage:
    print("\nMasked {}% of nodes\n".format(int(m*100)))
    loss, op = train(adj,feature,labels, m)


Masked 30% of nodes

Epoch: 0001 Loss: 1.19174826 time: 0.0040s
Epoch: 1001 Loss: 1.19036651 time: 0.0020s
Epoch: 2001 Loss: 1.18848038 time: 0.0010s
Epoch: 3001 Loss: 1.18620360 time: 0.0020s
Epoch: 4001 Loss: 1.18365431 time: 0.0010s
Epoch: 5001 Loss: 1.18108559 time: 0.0010s
Epoch: 6001 Loss: 1.17843926 time: 0.0010s
Epoch: 7001 Loss: 1.17549932 time: 0.0010s
Epoch: 8001 Loss: 1.17254174 time: 0.0022s
Epoch: 9001 Loss: 1.16946137 time: 0.0020s
Optimization Finished!
Total time elapsed: 12.9567s

Masked 50% of nodes

Epoch: 0001 Loss: 1.31260371 time: 0.0020s
Epoch: 1001 Loss: 1.30969441 time: 0.0030s
Epoch: 2001 Loss: 1.30616581 time: 0.0020s
Epoch: 3001 Loss: 1.30207932 time: 0.0020s
Epoch: 4001 Loss: 1.29741192 time: 0.0020s
Epoch: 5001 Loss: 1.29252911 time: 0.0010s
Epoch: 6001 Loss: 1.28740168 time: 0.0010s
Epoch: 7001 Loss: 1.28184080 time: 0.0020s
Epoch: 8001 Loss: 1.27599096 time: 0.0021s
Epoch: 9001 Loss: 1.26960981 time: 0.0020s
Optimization Finished!
Total time elapsed: 1